### Basic processing of Samsung data from Azure cameras

It includes undistortion of color and depth images and reprojection of depth image to undistrorted color virtual camera.

Parameters to set: `SEQUENCE_PATH` - path for recorded sequence, `OUTPUT_DIR` - where postprocessed data will be located.

**Attention**: calibration file `calib_params.yaml` for every camera should be located in original camera data directories.

In [1]:
import cv2
import imageio
import os
import shutil

from tqdm import tqdm

from utils.calib_io_utils import load_azure_params
from utils.cv_utils import undistort_image, reproject_depth

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2022-06-18 19:32:50,469 - utils - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 2022-06-18 19:32:50,470 - utils - NumExpr defaulting to 8 threads.


In [6]:
SEQUENCE_PATH = '/home/anasyasiia/Downloads/2022-04-16-12-15-47_round0/2022-04-16-12-15-47_round0/'
OUTPUT_DIR = 'output'

### List recorded cameras

In [7]:
camera_names = os.listdir(SEQUENCE_PATH)
camera_names

['3m', '5s']

### Prepare infrastructure for postprocessed data

In [8]:
dataset_basedir = os.path.basename(os.path.normpath(SEQUENCE_PATH))
postprocessed_data_dir = os.path.join(OUTPUT_DIR, dataset_basedir)

if os.path.exists(postprocessed_data_dir):
    shutil.rmtree(postprocessed_data_dir)
    
os.mkdir(postprocessed_data_dir)

for camera_name in camera_names:
    os.mkdir(os.path.join(postprocessed_data_dir, camera_name))
    os.mkdir(os.path.join(postprocessed_data_dir, camera_name, 'color'))
    os.mkdir(os.path.join(postprocessed_data_dir, camera_name, 'depth'))

### Iterate over cameras and do undistortion and reprojection 

In [ ]:
for camera_name in camera_names:
    camera_data_path = os.path.join(SEQUENCE_PATH, camera_name)
    azure_calib_path = os.path.join(camera_data_path, 'calib_params.json')

    color_camera, depth_camera, T_depth2color = load_azure_params(azure_calib_path)

    color_images_dir = os.path.join(camera_data_path, 'color')
    color_image_names = os.listdir(color_images_dir)
    color_image_names.sort()
    postprocessed_color_dir = os.path.join(postprocessed_data_dir, camera_name, 'color')

    for color_image_name in tqdm(color_image_names):
        image = cv2.imread(os.path.join(color_images_dir, color_image_name), -1)
        undistorted_image = undistort_image(image, color_camera)
        imageio.imwrite(os.path.join(postprocessed_color_dir, color_image_name), image)

    depth_images_dir = os.path.join(camera_data_path, 'depth')
    depth_image_names = os.listdir(depth_images_dir)
    depth_image_names.sort()
    postprocessed_depth_dir = os.path.join(postprocessed_data_dir, camera_names, 'depth')

    for depth_image_name in tqdm(depth_image_names):
        image = cv2.imread(os.path.join(depth_images_dir, depth_image_name), -1)
        undistorted_image = undistort_image(image, depth_camera)
        reprojected_image = reproject_depth(undistorted_image, depth_camera, color_camera, T_depth2color)
        imageio.imwrite(os.path.join(postprocessed_depth_dir, depth_image_name), reprojected_image)

  7%|▋         | 114/1699 [01:50<44:15,  1.68s/it]